In [1]:
import sys
sys.path.insert(1,'../tools')
from transformers import modeling_bert
import numpy as np
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from tqdm import tqdm
import pdb
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

model_name = 'Clinical_Bert'
import time
import utils
import imp
#from utils import batch_sent_loader
import os
from sklearn.metrics import confusion_matrix
import json
import pickle

In [2]:
class DocDataset(Dataset):
    def __init__(self, Doc_list, labels):
        #'Initialization'
        self.labels = labels
        self.Doc_list = Doc_list
    def __len__(self):
        #'Denotes the total number of samples'
        return len(self.Doc_list)
    def __getitem__(self, index):
        #'Generates one sample of data'
        # Load data and get label
        X = self.Doc_list[index]
        y = self.labels[index]
        return X, y
    
def load_data(shuffle_train = True):
    src_dir = '/home/zilong.zhang1/AE_automation/preprocessing/result/pu_chart/'
    text_train = pickle.load(open(src_dir+'text_train_se_merged.pkl','rb'))
    text_test = pickle.load(open(src_dir+'text_test_se_merged.pkl','rb'))
    label_test = pickle.load(open(src_dir+'Y_test.pkl','rb'))
    label_train = pickle.load(open(src_dir+'Y_train.pkl','rb'))    
    params_sent = {'batch_size': 1,
        'shuffle': shuffle_train,
        'num_workers': 6}
    params_sent_validation = {'batch_size': 1,
            'shuffle': False,
            'num_workers': 6}
    training_set = DocDataset(text_train, label_train)
    training_generator = DataLoader(training_set, **params_sent)

    validation_set = DocDataset(text_test, label_test)
    validation_generator =DataLoader(validation_set, **params_sent_validation)
    dummy_set = DocDataset(text_train[:5], label_train[:5])
    dummy_generator = DataLoader(dummy_set, **params_sent)
    return training_generator,validation_generator,dummy_generator

In [3]:
imp.reload(utils)
hpara1 = utils.hpara()
hpara1.ratio = 2
hpara1.batch_size = 4
hpara1.word_layers=4
hpara1.sent_layers=1
hpara1.use_position_embedding = False
hpara1.word_lr = 1e-04 #5e-05
hpara1.sent_lr = 8e-04 #2e-04
hpara1.max_sent_len = 100
hpara1.fix_cls = True
hpara1.head_num = 12
hpara1.max_doc_len = 300
hpara1.use_SSI_Bert = False
hpara1.use_narrow = False

In [6]:
count = 0
for aaa in training_generator:
    count+=1
print(count)

6119


In [10]:
pretrain_model_dir = '../BERT/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000'
vocab_file = os.path.join(pretrain_model_dir,'vocab.txt')
bert_config_file = os.path.join(pretrain_model_dir,'bert_config.json')
print(bert_config_file)
training_generator,validation_generator,dummy_generator = load_data()
tokenizer = utils._load_tf_tokenizer(vocab_file = vocab_file)

config = modeling_bert.BertConfig.from_json_file(bert_config_file)
config.num_hidden_layers = hpara1.word_layers
config.output_attentions = True
model_word = modeling_bert.BertModel(config)

import re
model_state_dict = pretrain_model_dir + '/pytorch_model.bin'
pretrained_dict = torch.load(model_state_dict)
model_dict = model_word.state_dict()
matched_dict = {}
for k in pretrained_dict.keys():
    try:
        new_k = re.search(r'(bert\.)(.*)',k).group(2)
    except:
        continue
    if new_k in model_dict:
        matched_dict[new_k] = pretrained_dict[k]
model_dict.update(matched_dict)
model_word.load_state_dict(model_dict)
cls_weight = model_word.state_dict()['embeddings.word_embeddings.weight'][101]


config_doc = modeling_bert.BertConfig.from_json_file(bert_config_file)
config_doc.num_hidden_layers = hpara1.sent_layers
config_doc.output_attentions = True
config_doc.num_attention_heads = hpara1.head_num
use_advanced_loss = hpara1.use_angular
use_position_embedding = hpara1.use_position_embedding
model_sent = modeling_bert.BertModel_no_embedding(config_doc,cls_weight,use_position_embedding=use_position_embedding)
print(config_doc)

model_word = model_word.cuda()
model_sent = model_sent.cuda()


../BERT/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000/bert_config.json
BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 1,
  "output_attentions": true,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



In [11]:
date = str(list(time.localtime())[0:3]).replace(', ','_')
save_dir = './exp/'+model_name+'_'+date[1:-1]+'/'

if not os.path.exists(save_dir):
    cmd = 'mkdir -p ' + save_dir
    os.system(cmd)
     
with open(save_dir + 'hpara.json', 'w') as fp:
    json.dump(hpara1.__dict__, fp)
    
do_train = True
do_test = True

max_epoch = hpara1.max_epoch
log = 'Iter {}/{}, Loss={:.3f},accu={:.3f},time={:.3f}\n'
from tqdm import tqdm
batch_size = hpara1.batch_size
accumulation_steps = hpara1.accumulation_steps
max_sent_len = hpara1.max_sent_len
max_doc_len = hpara1.max_doc_len
para_dict = {}
hpara_list = []

In [12]:
utils.model_train_and_test(hpara1,model_word,model_sent,save_dir,training_generator,validation_generator,tokenizer=tokenizer,narrow=hpara1.use_narrow)

0
1000
2000
3000
4000
5000
6000
Iter 0/60, Loss=4031.912,accu=0.661,time=1156.081

Test Iter 0/60, Loss=790.671,accu=0.832,auc=0.877,time=150.878

[[910 132]
 [125 363]]
0
1000
2000
3000
4000
5000
6000
Iter 1/60, Loss=2064.083,accu=0.859,time=1157.550

Test Iter 1/60, Loss=470.877,accu=0.861,auc=0.945,time=150.762

[[869 173]
 [ 40 448]]
0
1000
2000
3000
4000
5000
6000
Iter 2/60, Loss=1720.923,accu=0.888,time=1157.711

Test Iter 2/60, Loss=462.805,accu=0.861,auc=0.952,time=151.249

[[868 174]
 [ 38 450]]
0
1000
2000
3000
4000
5000
6000
Iter 3/60, Loss=1542.031,accu=0.897,time=1158.731

Test Iter 3/60, Loss=420.614,accu=0.888,auc=0.955,time=150.972

[[919 123]
 [ 49 439]]
0
1000
2000
3000
4000
5000
6000
Iter 4/60, Loss=1360.793,accu=0.915,time=1157.756

Test Iter 4/60, Loss=403.933,accu=0.893,auc=0.954,time=151.244

[[952  90]
 [ 73 415]]
0
1000
2000
3000
4000
5000
6000
Iter 5/60, Loss=1192.850,accu=0.923,time=1157.012

Test Iter 5/60, Loss=439.929,accu=0.902,auc=0.954,time=151.049

[[9

Process Process-220:
Traceback (most recent call last):
  File "/home/zilong.zhang1/miniconda3/envs/py37/lib/python3.7/multiprocessing/util.py", line 337, in _exit_function
    _run_finalizers()
  File "/home/zilong.zhang1/miniconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/home/zilong.zhang1/miniconda3/envs/py37/lib/python3.7/multiprocessing/util.py", line 277, in _run_finalizers
    finalizer()
  File "/home/zilong.zhang1/miniconda3/envs/py37/lib/python3.7/multiprocessing/util.py", line 201, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/zilong.zhang1/miniconda3/envs/py37/lib/python3.7/multiprocessing/queues.py", line 192, in _finalize_join
    thread.join()
  File "/home/zilong.zhang1/miniconda3/envs/py37/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/home/zilong.zhang1/miniconda3/envs/py37/lib/python3.7/threading.py", line 1060, in _wait

KeyboardInterrupt: 